## SVARpy: Block-Recursive SVAR and SVAR-GMM

This file shows how to pass block-recursive restrictions to the `SVAR.SVARest(u, estimator)` function. 
Imposing a block-recursive order allows to relax the non-Gaussianity and independence assumptions, 
simplifies labeling, and improves estimation performance, see Keweloh et al. (2023).
 

In [3]:
# Install required packages

# If required pip install SVARpy package
if True:
    !pip install SVARpy

import numpy as np
import matplotlib.pyplot as plt
import SVAR as SVAR
np.random.seed(0)

## Simulate the SVAR

Simulate reduced form shocks $u_t$ from the SVAR

$$
u_t = B_0 ɛ_t
$$

 with $n$ variables, $T$ observations, and structural shocks $ɛ_t$ generated from a uniform distribution.

In [4]:
n = 4  # Number of variables
T = 250  # Number of observations
# Specitfy B0
B0 = np.eye(n)
# Draw structural shocks
eps = np.empty([T, n])
for i in range(n):
    eps[:, i] = np.random.uniform(low=-np.sqrt(3), high=np.sqrt(3), size=T).T
# Generate reduced form shocks u = B eps
u = np.matmul(B0, np.transpose(eps))
u = np.transpose(u)

## Block-Recursive SVAR

A block-recursive structure can be passed to all estimstors using the option `prepOptions['blocks']`.
In the example below, the SVAR is ordered into two blocks, where the first block contains the first variable and the second block the remaining variables.

In [5]:
# Specify block-recursive structure
block1 = np.array([1, 1])
block2 = np.array([2, n])
blocks = list()
blocks.append(block1)
blocks.append(block2)

# Estimate SVAR with block-recursive structure and SVAR-GMM estimator
prepOptions = dict()
prepOptions['blocks'] = blocks 
SVAR_out = SVAR.SVARest(u, estimator='GMM', prepOptions=prepOptions)

# Estimate SVAR with block-recursive structure and fast SVAR-GMM estimator
prepOptions = dict()
prepOptions['blocks'] = blocks 
SVAR_out = SVAR.SVARest(u, estimator='GMM_WF', prepOptions=prepOptions)

# Estimate SVAR with block-recursive structure and SVAR-CUE estimator
prepOptions = dict()
prepOptions['blocks'] = blocks 
SVAR_out = SVAR.SVARest(u, estimator='CUE', prepOptions=prepOptions)


Estimator: GMM (steps=2)
Estimator Wopt/Avar: Uncorrelated/Uncorrelated
| SVAR            | Moments      | Tests             | E[e_1^m] / ... / E[e_n^m]        |
|-----------------|--------------|-------------------|----------------------------------|
| T=250           | #second: 10  | WaldRec=11.09     | m=2: 1.0 / 1.0 / 1.03 / 1.01     |
| n=4             | #third: 7    | WaldRec-pval=0.01 | m=3: -0.08 / 0.11 / -0.09 / 0.13 |
| #restrictions:3 | #fourth: 12  | J=13.37           | m=4: 1.87 / 1.73 / 1.93 / 1.76   |
| #unknowns:13    | ->loss: 0.05 | J-pval=0.65       |                                  |
 
|                  |       B(:,1)       |       B(:,2)       |       B(:,3)       |       B(:,4)       |
|------------------|--------------------|--------------------|--------------------|--------------------|
|      B(1,:)      |        0.98        |       *0.0*        |       *0.0*        |       *0.0*        |
| (avar/wald/pval) | (0.19/1236.03/0.0) |   (nan/nan/nan)    |   (nan/n

The examples below provide additional illustrations of different block-recursive structures. It is important to note that when a finer block-recursive structure is imposed, the number of coskewness and cokurtosis conditions utilized by the SVAR-GMM estimator automatically decreases. For instance, without any restrictions, the estimator employs 16 coskewness and 31 cokurtosis conditions. However, when utilizing four blocks (a recursive SVAR), the estimator does not employ any coskewness or cokurtosis conditions, rendering it equivalent to the typical recursive SVAR estimator.

In [6]:
# Specify block-recursive structure
block1 = np.array([1, n]) 
blocks = list()
blocks.append(block1) 
# Estimate SVAR with block-recursive structure
prepOptions = dict()
prepOptions['blocks'] = blocks 
SVAR_out = SVAR.SVARest(u, estimator='GMM', prepOptions=prepOptions)

# Specify block-recursive structure
block1 = np.array([1, 2])
block2 = np.array([3, n])
blocks = list()
blocks.append(block1)
blocks.append(block2)
# Estimate SVAR with block-recursive structure
prepOptions = dict()
prepOptions['blocks'] = blocks 
SVAR_out = SVAR.SVARest(u, estimator='GMM', prepOptions=prepOptions)
 


# Specify block-recursive structure
block1 = np.array([1, 1])
block2 = np.array([2, 2])
block3 = np.array([3, n])
blocks = list()
blocks.append(block1)
blocks.append(block2)
blocks.append(block3)
# Estimate SVAR with block-recursive structure
prepOptions = dict()
prepOptions['blocks'] = blocks 
SVAR_out = SVAR.SVARest(u, estimator='GMM', prepOptions=prepOptions)

# Specify block-recursive structure
block1 = np.array([1, 1])
block2 = np.array([2, 2])
block3 = np.array([3, 3])
block4 = np.array([4, n])
blocks = list()
blocks.append(block1)
blocks.append(block2)
blocks.append(block3)
blocks.append(block4)
# Estimate SVAR with block-recursive structure
prepOptions = dict()
prepOptions['blocks'] = blocks 
SVAR_out = SVAR.SVARest(u, estimator='GMM', prepOptions=prepOptions)

Estimator: GMM (steps=2)
Estimator Wopt/Avar: Uncorrelated/Uncorrelated
| SVAR            | Moments      | Tests             | E[e_1^m] / ... / E[e_n^m]        |
|-----------------|--------------|-------------------|----------------------------------|
| T=250           | #second: 10  | WaldRec=18.37     | m=2: 1.03 / 0.97 / 1.05 / 0.99   |
| n=4             | #third: 16   | WaldRec-pval=0.01 | m=3: -0.08 / 0.11 / -0.09 / 0.13 |
| #restrictions:0 | #fourth: 31  | J=45.2            | m=4: 1.97 / 1.61 / 2.02 / 1.7    |
| #unknowns:16    | ->loss: 0.18 | J-pval=0.3        |                                  |
 
|                  |      B(:,1)       |      B(:,2)       |       B(:,3)       |       B(:,4)       |
|------------------|-------------------|-------------------|--------------------|--------------------|
|      B(1,:)      |       0.97        |       0.06        |       -0.06        |        0.02        |
| (avar/wald/pval) | (0.17/1346.9/0.0) | (0.25/3.28/0.07)  |  (0.28/2.79/0.1)

To retain the full set of moments used by the SVAR-GMM estimator, you can utilize the `moments_blocks=False` option. This feature suppresses the reduction in the number of moments employed by the SVAR-GMM estimator when imposing a block-recursive structure. 

In [7]:
# Specify block-recursive structure
block1 = np.array([1, n]) 
blocks = list()
blocks.append(block1) 
# Estimate SVAR with block-recursive structure
prepOptions = dict()
prepOptions['blocks'] = blocks 
prepOptions['moments_blocks'] = False
SVAR_out = SVAR.SVARest(u, estimator='GMM', prepOptions=prepOptions)

# Specify block-recursive structure
block1 = np.array([1, 2])
block2 = np.array([3, n])
blocks = list()
blocks.append(block1)
blocks.append(block2)
 # Estimate SVAR with block-recursive structure
prepOptions = dict()
prepOptions['blocks'] = blocks 
prepOptions['moments_blocks'] = False
SVAR_out = SVAR.SVARest(u, estimator='GMM', prepOptions=prepOptions)
 


# Specify block-recursive structure
block1 = np.array([1, 1])
block2 = np.array([2, 2])
block3 = np.array([3, n])
blocks = list()
blocks.append(block1)
blocks.append(block2)
blocks.append(block3)
# Estimate SVAR with block-recursive structure
prepOptions = dict()
prepOptions['blocks'] = blocks 
prepOptions['moments_blocks'] = False
SVAR_out = SVAR.SVARest(u, estimator='GMM', prepOptions=prepOptions)

# Specify block-recursive structure
block1 = np.array([1, 1])
block2 = np.array([2, 2])
block3 = np.array([3, 3])
block4 = np.array([4, n])
blocks = list()
blocks.append(block1)
blocks.append(block2)
blocks.append(block3)
blocks.append(block4)
# Estimate SVAR with block-recursive structure
prepOptions = dict()
prepOptions['blocks'] = blocks 
prepOptions['moments_blocks'] = False
SVAR_out = SVAR.SVARest(u, estimator='GMM', prepOptions=prepOptions)
 

Estimator: GMM (steps=2)
Estimator Wopt/Avar: Uncorrelated/Uncorrelated
| SVAR            | Moments      | Tests             | E[e_1^m] / ... / E[e_n^m]        |
|-----------------|--------------|-------------------|----------------------------------|
| T=250           | #second: 10  | WaldRec=18.37     | m=2: 1.03 / 0.97 / 1.05 / 0.99   |
| n=4             | #third: 16   | WaldRec-pval=0.01 | m=3: -0.08 / 0.11 / -0.09 / 0.13 |
| #restrictions:0 | #fourth: 31  | J=45.2            | m=4: 1.97 / 1.61 / 2.02 / 1.7    |
| #unknowns:16    | ->loss: 0.18 | J-pval=0.3        |                                  |
 
|                  |      B(:,1)       |      B(:,2)       |       B(:,3)       |       B(:,4)       |
|------------------|-------------------|-------------------|--------------------|--------------------|
|      B(1,:)      |       0.97        |       0.06        |       -0.06        |        0.02        |
| (avar/wald/pval) | (0.17/1346.9/0.0) | (0.25/3.28/0.07)  |  (0.28/2.79/0.1)

## References

Keweloh, Sascha Alexander. "A generalized method of moments estimator for structural vector autoregressions based on higher moments." Journal of Business & Economic Statistics 39.3 (2021): 772-782.

Keweloh, Sascha Alexander. "A feasible approach to incorporate information in higher moments in structural vector autoregressions." (2021b).

Keweloh, Sascha A., Stephan Hetzenecker, and Andre Seepe. "Monetary Policy and Information Shocks in a Block-Recursive SVAR." Journal of International Money and Finance (2023)